In [ ]:
### FOR COLAB ONLY ###

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

In [ ]:
#cd /content/drive/My Drive/ssnet

In [ ]:
#!pip install musdb

In [ ]:
### IMPORTS ###

In [ ]:
import musdb
import os
import torch
import torch.optim as optim
import torch.nn as nn
import pickle
import numpy as np
from torchmetrics.audio import SignalNoiseRatio, ScaleInvariantSignalNoiseRatio
from scipy.io.wavfile import write

In [ ]:
from modules.model import ssnet
from tqdm.notebook import tqdm
from modules.utils import transform
ssnet = ssnet()

In [ ]:
os.environ['MUSDB_PATH'] = "data/musdb18"

In [ ]:
mus_test = musdb.DB(subsets="test")

In [ ]:
frame_length=50000

In [ ]:
track = []
mixture_audio = mus_test[1].stems[0].T
for start_idx in tqdm(range(0, mixture_audio.shape[1] - frame_length + 1, frame_length)):
    # Extract the frames for mixture and target
    X = transform(mixture_audio[:, start_idx:start_idx+frame_length])
    pad = torch.zeros(2, 100)
    X = torch.cat([pad, X, pad], dim=1).view(1,2,-1)
    out = ssnet(X)[:,:,:,100:-100]
    track.append(out.detach())

In [ ]:
track1 = [x[:,0,:,:].numpy() for x in track]
track2 = [x[:,1,:,:].numpy() for x in track]
track3 = [x[:,2,:,:].numpy() for x in track]
track4 = [x[:,3,:,:].numpy() for x in track]

track1 = np.concatenate(track1, axis=2).reshape(2,-1).T
track2 = np.concatenate(track2, axis=2).reshape(2,-1).T
track3 = np.concatenate(track3, axis=2).reshape(2,-1).T
track4 = np.concatenate(track4, axis=2).reshape(2,-1).T

sample_rate = 44100 
#write('drums.wav', sample_rate, track1)
#write('bass.wav', sample_rate, track2)
#write('others.wav', sample_rate, track3)
#write('vocals.wav', sample_rate, track4)